In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [ ]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the data
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

In [ ]:
# Convert labels to numerical values
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

In [ ]:
# Preprocess the text data
vectorizer = TextVectorization(max_tokens=1000, output_mode='int')
text_ds = tf.data.Dataset.from_tensor_slices(train_data['message']).batch(128)
vectorizer.adapt(text_ds)
voc = vectorizer.get_vocabulary()

In [ ]:
# Build the model
model = keras.Sequential([
    vectorizer,
    layers.Embedding(len(voc), 64, mask_zero=True),
    layers.Dropout(0.5),
    layers.Conv1D(64, 5, padding='valid', activation='relu', strides=2),
    layers.GlobalMaxPooling1D(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_data['message'], train_data['label'], epochs=10, validation_split=0.2)

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data['message'], test_data['label'])
print('Test accuracy:', test_acc)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

    # Convert the message to a numpy array and reshape it
    msg_array = np.array([pred_text])
    msg_array = np.expand_dims(msg_array, axis=-1)

    # Use the model to predict the label probabilities
    label_probs = model.predict(msg_array)[0]
    if label_probs < 0.5:
      return [label_probs, 'ham']
    else:
        return [label_probs[0], 'spam']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
